# Gradient Boosting Regression From Scratch

Gradient Boosting is a powerful ensemble method that builds trees sequentially. Each new tree corrects the errors (residuals) made by the previous trees.

## Key Concepts:
- **Sequential Learning**: Trees are added one by one
- **Residuals**: The current error ($y - y_{pred}$)
- **Learning Rate (Shrinkage)**: Scaling factor for each tree's contribution
- **Additive Model**: $F_m(x) = F_{m-1}(x) + \eta h_m(x)$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

## 1. Algorithm Overview

1. Initialize prediction with mean: $F_0(x) = \text{mean}(y)$
2. For $m=1$ to $M$:
   a. Calculate residuals: $r_{im} = y_i - F_{m-1}(x_i)$
   b. Fit a weak learner $h_m(x)$ to the residuals $r_{im}$
   c. Update the model: $F_m(x) = F_{m-1}(x) + \eta \cdot h_m(x)$

In [ ]:
class GradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.lr = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.init_prediction = None

    def fit(self, X, y):
        # Step 1: Initial prediction (mean of y)
        self.init_prediction = np.mean(y)
        current_predictions = np.full(len(y), self.init_prediction)
        
        for _ in range(self.n_estimators):
            # Step 2a: Calculate residuals
            residuals = y - current_predictions
            
            # Step 2b: Fit tree to residuals
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            
            # Step 2c: Update predictions
            predictions = tree.predict(X)
            current_predictions += self.lr * predictions
            
            self.trees.append(tree)

    def predict(self, X):
        # Initial prediction
        y_pred = np.full(X.shape[0], self.init_prediction)
        
        # Add contributions from each tree
        for tree in self.trees:
            y_pred += self.lr * tree.predict(X)
            
        return y_pred

    def score(self, X, y):
        y_pred = self.predict(X)
        return 1 - np.sum((y - y_pred)**2) / np.sum((y - np.mean(y))**2)

## 2. Testing and Visualization

In [ ]:
X, y = make_regression(n_samples=200, n_features=1, noise=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=2)
gbr.fit(X_train, y_train)
print(f"Our Gradient Boosting R2: {gbr.score(X_test, y_test):.4f}")

X_line = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
plt.scatter(X, y, alpha=0.5)
plt.plot(X_line, gbr.predict(X_line), color='red', linewidth=3)
plt.title("Gradient Boosting Fit")
plt.show()